### Fact Orders

### Data Reading

In [0]:
df = spark.sql("select * from databricks_cata.silver.orders_silver")

In [0]:
df.display()

In [0]:
df_dimcus = spark.sql("""
    SELECT
        DimCustomerKey,
        customer_id AS dim_customer_id
    FROM databricks_cata.gold.dimcustomers
""")

df_dimpro = spark.sql("""
    SELECT
        product_id AS DimProductKey,
        product_id AS dim_product_id
    FROM databricks_cata.gold.dimproducts
""")


### Fact Dataframe

In [0]:
df_fact = (
    df.join(
        df_dimcus,
        df["customer_id"] == df_dimcus["dim_customer_id"],
        how="left"
    )
    .join(
        df_dimpro,
        df["product_id"] == df_dimpro["dim_product_id"],
        how="left"
    )
)

df_fact_new = df_fact.drop("dim_customer_id", "dim_product_id",'customer_id','product_id')




### upsert in fact table

In [0]:
from delta.tables import DeltaTable

In [0]:


if spark.catalog.tableExists("databricks_cata.gold.FactOrders"):

    dlt_obj = DeltaTable.forName(
        spark,
        "databricks_cata.gold.FactOrders"
    )

    (
        dlt_obj.alias("trg")
        .merge(
            df_fact_new.alias("src"),
            """
            trg.order_id = src.order_id
            AND trg.DimCustomerKey = src.DimCustomerKey
            AND trg.DimProductKey = src.DimProductKey
            """
        )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )

else:

    (
        df_fact_new.write
        .format("delta")
        .option(
            "path",
            "abfss://gold@databricksetenimit.dfs.core.windows.net/FactOrders"
        )
        .saveAsTable("databricks_cata.gold.FactOrders")
    )


In [0]:
%sql
select * from databricks_cata.gold.FactOrders